<a href="https://colab.research.google.com/github/MarcosVeniciu/HotelQA-RAG/blob/main/Fine_tuning_code/FineTuningTinyLlama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes

In [2]:
from transformers.trainer_utils import get_last_checkpoint
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
from unsloth import FastLanguageModel
from datasets import load_dataset
from trl import SFTTrainer
import torch
import os

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [3]:
project_path = "TinyLlama_T" # Nome da pasta do projeto
salvar_GDrive = True # Se True, salva os checkpoints no Google Drive

In [4]:
if salvar_GDrive:
  # Mount Google Drive
  from google.colab import drive
  drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.


model, tokenizer = FastLanguageModel.from_pretrained(
  model_name = "unsloth/tinyllama-bnb-4bit",
  max_seq_length = max_seq_length,
  dtype = dtype,
  load_in_4bit = load_in_4bit,
)

==((====))==  Unsloth: Fast Llama patching release 2024.8
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.26.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [6]:
model = FastLanguageModel.get_peft_model(
  model,
  r = 128, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
  target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj",],
  lora_alpha = 32,
  lora_dropout = 0, # Currently only supports dropout = 0
  bias = "none",    # Currently only supports bias = "none"
  use_gradient_checkpointing = False, # @@@ IF YOU GET OUT OF MEMORY - set to True @@@
  random_state = 3407,
  use_rslora = False,  # We support rank stabilized LoRA
  loftq_config = None, # And LoftQ
)

Unsloth 2024.8 patched 22 layers with 22 QKV layers, 22 O layers and 22 MLP layers.


In [7]:
ALPACA_PROMPT_DICT = {
  "prompt_context": (
    "{bos_token}Below is an instruction that describes a task, paired with an input that provides further context."
    "Write a response that appropriately completes the request."
    "Write a response between <res> and </res>.\n\n"
    "### Instruction:\n"
    "{instruction}\n\n"
    "### Input:\n"
    "{input}\n\n"
    "### Response:\n"
    "<res>{response}</res>{eos_token}"
  )
}

BOS_TOKEN = tokenizer.bos_token
EOS_TOKEN = tokenizer.eos_token
def formatPrompt(example):
  prompt = ALPACA_PROMPT_DICT['prompt_context'].format(
    bos_token=BOS_TOKEN,
    instruction=f"respond: {example['Question']}",
    input=f"context: {example['Context']}",
    response=example['Answer'],
    eos_token=EOS_TOKEN
  )

  return {"text": prompt}


# downlad dataset
if not os.path.exists('Dataset_V2_train_16k.parquet'):
  !wget https://github.com/MarcosVeniciu/HotelQA-RAG/raw/main/Dataset/16k/Dataset_V2_train_16k.parquet
  !wget https://github.com/MarcosVeniciu/HotelQA-RAG/raw/main/Dataset/16k/Dataset_V2_test_16k.parquet

dataset = load_dataset('parquet', data_files={'train': 'Dataset_V2_train_16k.parquet',
                                              'test': 'Dataset_V2_test_16k.parquet'})
# Aplicando a função formatPrompt ao dataset
dataset = dataset.map(formatPrompt, remove_columns=['Context', 'Question', 'Answer'])

In [8]:
if os.path.exists("/content/drive"):
  project_dir = os.path.join("/content/drive/MyDrive/Treinamento", project_path)
else:
  project_dir = project_path
print(f"Project diretory: {project_dir}")

num_epocas = 9

trainer = SFTTrainer(
  model = model,
  tokenizer = tokenizer,
  train_dataset = dataset['train'].shuffle(),
  dataset_text_field = "text",
  max_seq_length = max_seq_length,
  dataset_num_proc = 2,
  packing = True, # Packs short sequences together to save time!
  args = TrainingArguments(
    num_train_epochs = num_epocas,
    per_device_train_batch_size = 4,
    gradient_accumulation_steps = 3,
    save_total_limit=2,
    save_strategy="steps",
    save_steps=50,
    warmup_ratio = 0.4,
    learning_rate = 2e-5,
    fp16 = not is_bfloat16_supported(),
    bf16 = is_bfloat16_supported(),
    logging_steps = 10,
    optim = "adamw_8bit",
    weight_decay = 0.1,
    lr_scheduler_type = "cosine",
    seed = 3407,
    output_dir=project_dir,
  ),
)

Project diretory: /content/drive/MyDrive/Treinamento/TinyLlama_T


In [9]:
#se tiver algum checkpoint salvo na pasta do projeto, ele vai continuar a partir do ultimo salvo.
last_checkpoint = get_last_checkpoint(project_dir)
if last_checkpoint != None: # Continua a partir do ultimo checkpoint salvo
  print(f"Continuando treinamento a partir de: {last_checkpoint}\n")
  trainer_stats = trainer.train(resume_from_checkpoint=last_checkpoint)
else: # começa um novo treinamento
  print("Começando um novo treinamento:")
  trainer_stats = trainer.train()

trainer.save_model(os.path.join(project_dir, "gemma_ft_saved"))

Continuando treinamento a partir de: /content/drive/MyDrive/Treinamento/TinyLlama_T/checkpoint-560



==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 1,344 | Num Epochs = 9
O^O/ \_/ \    Batch size per device = 4 | Gradient Accumulation steps = 3
\        /    Total batch size = 12 | Total steps = 1,008
 "-____-"     Number of trainable parameters = 100,925,440


Step,Training Loss
570,0.220600
580,0.220800
590,0.217200
600,0.224500
610,0.217700
620,0.209800
630,0.207500
640,0.215600
650,0.205500
660,0.211600


Step,Training Loss
570,0.220600
580,0.220800
590,0.217200
600,0.224500
610,0.217700
620,0.209800
630,0.207500
640,0.215600
650,0.205500
660,0.211600


**Coisas a serem observadas**

1 teste \
apos eu ttreinar por 26 epocas, respondeu mais ou menos bem para a tarefa, tanto quando o modelo usado era o mesmo que foi treinado, como o qando carrega ele a partir do modelo salvo. \
Mas ele perdeu a capacidade de responder a outros tipos de tarefas.
Quando eu carrego o modelo a partir do modelo_base e mesclo ele com o modelo salvo, ele da uma resposta curta e fica repetindo todo o prompt em loop, porem ele não perdeu a capacidade de resolver outras tarefas.

--------------------------

2 teste \
Para o segundo teste, vou manter o uso do checkpoint, porem vou mudar a forma como uso os tokens s e /s, para ver ser isso mouda algo.


In [10]:
contexto = "Hotel Fasano, São Paulo. This luxury hotel, example located in the Jardins neighborhood, offers elegant accommodations with stunning city views. The hotel features a pool, spa and wellness center, gourmet restaurant, fitness center, and 24-hour reception."
pergunta = "Does Hotel Fasano have a pool available for guests?"
pergunta2 = "Does Hotel Fasano have a free wi-fi available for guests?"

ALPACA_PROMPT_DICT = {
  "prompt_context": (
    "Below is an instruction that describes a task, paired with an input that provides further context."
    "Write a response that appropriately completes the request."
    "Write a response between <res> and </res>.\n\n"
    "### Instruction:\n"
    "{instruction}\n\n"
    "### Input:\n"
    "{input}\n\n"
    "### Response:\n"
    "<res>"
  )
}


In [ ]:
# Inferencia com o modelo apos o treinamento
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
  ALPACA_PROMPT_DICT['prompt_context'].format(
  instruction=f"respond: {pergunta}",
  input=f"context: {contexto}")

], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 250)

In [11]:
# Inferencia com o modelo apos o treinamento
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
  ALPACA_PROMPT_DICT['prompt_context'].format(
  instruction=f"respond: {pergunta2}",
  input=f"context: {contexto}")

], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 250)

<s> Below is an instruction that describes a task, paired with an input that provides further context.Write a response that appropriately completes the request.Write a response between <res> and </res>.

### Instruction:
respond: Does Hotel Fasano have a free wi-fi available for guests?

### Input:
context: Hotel Fasano, São Paulo. This luxury hotel, example located in the Jardins neighborhood, offers elegant accommodations with stunning city views. The hotel features a pool, spa and wellness center, gourmet restaurant, fitness center, and 24-hour reception.

### Response:
<res>Yes, Hotel Fasano, São Paulo offers a free Wi-Fi access available for guests, ensuring that they stay connected during their stay.</res></s>


In [12]:
# Inferencia com o modelo apos o treinamento
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
  ALPACA_PROMPT_DICT['prompt_context'].format(
    instruction="Continue the fibonnaci sequence.",
    input=f"1, 1, 2, 3, 5, 8"    )

], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 250)

<s> Below is an instruction that describes a task, paired with an input that provides further context.Write a response that appropriately completes the request.Write a response between <res> and </res>.

### Instruction:
Continue the fibonnaci sequence.

### Input:
1, 1, 2, 3, 5, 8

### Response:
<res>Although the hotel does not have a pool, it does offer a fitness center and a restaurant with bar for guests to relax and unwind. The hotel features a parking lot and wheelchair accessibility.</res></s>


In [13]:
# Inferencia com o modelo carregando o modelo salvo
if True:
  from unsloth import FastLanguageModel
  modelo, tokenizero = FastLanguageModel.from_pretrained(
    model_name = "/content/drive/MyDrive/Treinamento/TinyLlama_T/gemma_ft_saved", # YOUR MODEL YOU USED FOR TRAINING
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
  )
  FastLanguageModel.for_inference(modelo) # Enable native 2x faster inference


inputs = tokenizero(
[
  ALPACA_PROMPT_DICT['prompt_context'].format(
  instruction=f"respond: {pergunta}",
  input=f"context: {contexto}")
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizero)
_ = modelo.generate(**inputs, streamer = text_streamer, max_new_tokens = 120)

==((====))==  Unsloth: Fast Llama patching release 2024.8
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.26.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
<s> Below is an instruction that describes a task, paired with an input that provides further context.Write a response that appropriately completes the request.Write a response between <res> and </res>.

### Instruction:
respond: Does Hotel Fasano have a pool available for guests?

### Input:
context: Hotel Fasano, São Paulo. This luxury hotel, example located in the Jardins neighborhood, offers elegant accommodations with stunning city views. The hotel features a pool, spa and wellness center, gourmet restaurant, fitness center, and 24-hour reception.

### Response

In [14]:
# Inferencia com o modelo a partir do modelo salvo
FastLanguageModel.for_inference(modelo) # Enable native 2x faster inference
inputs = tokenizero(
[
  ALPACA_PROMPT_DICT['prompt_context'].format(
    instruction="Continue the fibonnaci sequence.",
    input=f"1, 1, 2, 3, 5, 8"    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizero)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<s> Below is an instruction that describes a task, paired with an input that provides further context.Write a response that appropriately completes the request.Write a response between <res> and </res>.

### Instruction:
Continue the fibonnaci sequence.

### Input:
1, 1, 2, 3, 5, 8

### Response:
<res>Although the hotel does not have a pool, it does offer a fitness center and a restaurant with bar for guests to relax and unwind. The hotel features a parking lot and wheelchair accessibility.</res></s>


In [15]:
# Inferencia com o modelo, mesclando o modelo base com o modelo salvo
from peft import get_peft_model, PeftConfig

# Carregar o modelo base
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.


modele, tokenizere = FastLanguageModel.from_pretrained(
  model_name = "unsloth/tinyllama-bnb-4bit",
  max_seq_length = max_seq_length,
  dtype = dtype,
  load_in_4bit = load_in_4bit,
)

# Carregar os adaptadores LoRA
lora_path = "/content/drive/MyDrive/Treinamento/TinyLlama_T/gemma_ft_saved"
# Carregar a configuração dos adaptadores LoRA
peft_config = PeftConfig.from_pretrained(lora_path)

# Combinar o modelo base com os adaptadores LoRA
modele = get_peft_model(modele, peft_config)
FastLanguageModel.for_inference(modele) # Enable native 2x faster inference


==((====))==  Unsloth: Fast Llama patching release 2024.8
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.26.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [16]:
# Inferencia com o modelo apos o treinamento
FastLanguageModel.for_inference(modele) # Enable native 2x faster inference
inputs = tokenizere(
[
  ALPACA_PROMPT_DICT['prompt_context'].format(
  instruction=f"respond: {pergunta}",
  input=f"context: {contexto}")
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizere)
_ = modele.generate(**inputs, streamer = text_streamer, max_new_tokens = 120)

<s> Below is an instruction that describes a task, paired with an input that provides further context.Write a response that appropriately completes the request.Write a response between <res> and </res>.

### Instruction:
respond: Does Hotel Fasano have a pool available for guests?

### Input:
context: Hotel Fasano, São Paulo. This luxury hotel, example located in the Jardins neighborhood, offers elegant accommodations with stunning city views. The hotel features a pool, spa and wellness center, gourmet restaurant, fitness center, and 24-hour reception.

### Response:
<res>

### Instruction:
respond: What is the name of the hotel?

### Input:
context: Hotel Fasano, São Paulo. This luxury hotel, example located in the Jardins neighborhood, offers elegant accommodations with stunning city views. The hotel features a pool, spa and wellness center, gourmet restaurant, fitness center, and 24-hour reception.

### Response:
<res>

### Instruction:
respond: What is the name of the hotel?

##


In [17]:
# Inferencia com o modelo apos o treinamento
FastLanguageModel.for_inference(modele) # Enable native 2x faster inference
inputs = tokenizere(
[
  ALPACA_PROMPT_DICT['prompt_context'].format(
    instruction="Continue the fibonnaci sequence.",
    input=f"1, 1, 2, 3, 5, 8"    )

], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizere)
_ = modele.generate(**inputs, streamer = text_streamer, max_new_tokens = 250)

<s> Below is an instruction that describes a task, paired with an input that provides further context.Write a response that appropriately completes the request.Write a response between <res> and </res>.

### Instruction:
Continue the fibonnaci sequence.

### Input:
1, 1, 2, 3, 5, 8

### Response:
<res>1</res>
<res>2</res>
<res>3</res>
<res>4</res>
<res>5</res>
<res>6</res>
<res>7</res>
<res>8</res>
<res>9</res>
<res>10</res>
<res>11</res>
<res>12</res>
<res>13</res>
<res>14</res>
<res>15</res>
<res>16</res>
<res>17</res>
<res>18</res>
<res>19</res>
<res>20</res>
<res>21</res>
<res>22</res>
<res>23</res>
<res>24</res>
<res>25</res>
<res>26</res>
<res>27</res>
<res>28</res>
<res>29</res>
<


**REFERENCIAS**

[1] [Colab original com tinyLlama](https://colab.research.google.com/drive/1AZghoNBQaMDgWJpi4RbffGM1h6raLUj9?usp=sharing#scrollTo=95_Nn-89DhsL)